# Introduction
- Présentation du sujet : Simulation numérique d'une équation aux dérivées partielles (EDP) de convection
- Explication du but de la simulation : Étudier l'évolution d'un champ de densité soumis à une convection

## Equation de Burgers
L'équation de Burgers est une équation aux dérivées partielles issue de la mécanique des fluides. Elle apparaît dans divers domaines des mathématiques appliquées, comme la modélisation de la dynamique des gaz, de l'acoustique ou du trafic routier. Elle doit son nom à *Johannes Martinus Burgers* qui l'a discutée en 1948. Elle apparaît dans des travaux antérieurs de *Andrew Russel Forsyth* et *Harry Bateman*

### Formulations
En notant **u** la vitesse, et **ν** le coefficient de viscosité cinématique, la forme générale de [l'équation de Burgers](https://en.wikipedia.org/wiki/Burgers%27_equation) est:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2} $$ 

Quand $\nu =0$ , l'équation de Burgers devient l'équation de Burgers sans viscosité :

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0 $$

La matrice jacobienne de cette équation se réduit à la quantité scalaire u, valeur réelle. Il s'agit donc d'une équation aux dérivées partielles hyperbolique. Elle peut donc comporter des discontinuités (ondes de choc).

La forme conservative de cette équation est :

$$\frac{\partial u}{\partial t} + \frac{1}{2} \frac{\partial }{\partial x} (u^2)= 0 $$ 

# Les bibliothèques utilisées
La première partie du code importe la bibliothèque numpy et le module Output.
- **[NumPy](https://numpy.org/doc/stable/reference/)** : bibliothèque de calcul scientifique en Python
- **[Matplotlib](https://matplotlib.org/stable/index.html)** : bibliothèque pour la visualisation de données en Python
- Nous utiliserons aussi la fonction **my_animation** (permet de visualiser les images sauvegardées sous forme d'animation)

### Numpy et Matplotlib

In [4]:
import numpy as np
import matplotlib.pyplot as plt

### my_animation

In [5]:
import matplotlib.animation

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

def anim_add_figure(u, t, c_anim):
    c_anim[f"{t:07.3f}"] = u

def my_plot(time):
    plt.cla()
    plt.ylim((0.4,1.6))
    plt.plot(xn, c_anim[time],"b-o")
    plt.title(f"t={time}")
    plt.xlabel("x")
    plt.ylabel("u")
    
def my_animation(c_anim):
    fig, ax = plt.subplots()
    return matplotlib.animation.FuncAnimation(fig, my_plot, frames=list(c_anim.keys()))

# Les paramètres physiques
- On définit les paramètres physiques, notamment la *longueur du domaine* **L** et le *temps de simulation maximum* **tmax**.



In [6]:
L = 200.0 # m
tmax = 700 # s

# Les conditions aux limites
- Explication des conditions aux limites utilisées dans la simulation : **[Dirichlet](https://en.wikipedia.org/wiki/Dirichlet_boundary_condition)** ou **périodiques**:

Si Dirichlet est **True**, les conditions aux limites sont fixées pour être des conditions de Dirichlet (c'est-à-dire des valeurs fixes aux limites). 

Si Cyclic est **True**, les conditions aux limites sont définies pour être cycliques (c'est-à-dire que le domaine se replie sur lui-même).

In [7]:
Dirichlet = False
Cyclic = True
if Dirichlet:
    u_g = 0.0
    u_d = 0.0

### Exercice 1:
Voici un exercice pour mieux comprendre ce code:

- Modifier les conditions aux limites pour qu'elles soient de Dirichlet (fixées à zéro) plutôt que cycliques. 

Comment cela affecte-t-il les résultats de la simulation ?


In [8]:
# Réponse



# Les paramètres
## Paramètres Numériques
- Explication des paramètres numériques utilisés dans la simulation : nombre de points **Np**, nombre de Courant **CFL**

Avec:

**Np** le nombre de points de grille

**CFL** le nombre CFL , qui détermine la taille de pas de temps en fonction de la condition de *Courant-Friedrichs-Lewy*.

### Condition de Courant-Friedrichs-Lewy
La condition **[CFL](http://wikhydro.developpement-durable.gouv.fr/index.php/Courant-Friedrich-Levy_/_CFL_(condition_de)_(HU)#:~:text=Condition%20nécessaire%20pour%20qu%27un,%27équations%20aux%20dérivées%20partielles)** porte le nom de trois mathématiciens allemands (*Richard Courant, Kurt Friedrichs et Hans Lewy*) qui ont publié en 1928 un article (Courant et al., 1928) concernant l’analyse d’équations aux dérivées partielles et leur approximation numérique. Cet article mettait en évidence une condition nécessaire pour qu’un algorithme de calcul produise une solution *cohérente*. 

Cet article très en avance sur son temps a trouvé un intérêt majeur avec le développement des ordinateurs permettant la mise en œuvre de différentes méthodes numériques de résolution consistant à discrétiser le domaine de travail en pas de temps $\Delta{t}$ et en pas d'espace $\Delta{x}$.

Cette condition se met sous la forme :

$$\frac{\Delta{x}}{\Delta{t}} \geq c$$

Avec:

$c$ La célerité de l'onde (m/s)


In [9]:
# Numerical parameters
Np = 300
CFL = 0.8

### Exercice 2:

- Modifiez le nombre de points de grille Np. 

Que se passe-t-il lorsque vous diminuez Np ?

In [10]:
# Réponse



## Paramètre de Sortie 

Le paramètre de sortie **dtAff** est l'intervalle de sauvegarde des figures.

In [11]:
# Output parameters
dtAff = 5.0 # s

# Initialise other parameters
dx = L/Np # m
u = np.zeros(Np)
F = np.zeros(Np+1)
xn = np.linspace(dx/2,L-dx/2,Np)
t = 0

### Exercice 3:

- Essayez de changer les paramètres de la simulation tels que CFL ou dtAff. 

Comment cela affecte-t-il les résultats ?

In [12]:
# Réponse



# Les fonctions de flux
- Présentation des fonctions de flux utilisées dans la simulation : 'flux_upwind_conservative_naive', 'flux_upwind_conservative', 'flux_Godunov'

Ces trois fonctions de flux différentes : **`flux_upwind_conservative_naive`**, **`flux_upwind_conservative`** et **`flux_Godunov`** sont des fonctions qui calculent le flux de la quantité scalaire entre deux points adjacents dans le domaine.


In [13]:
# Def fluxes 
def flux_upwind_conservative_naive(U,V):
    uedge = (U + V)/2.0
    return (uedge > 0) * uedge * U + (uedge < 0) * uedge * V 

def flux_upwind_conservative(U,V):
    uedge = (U + V)/2.0
    return (uedge > 0) * U**2 + (uedge < 0) * V**2
   
def flux_Godunov(U,V):
    ustar = (U>=V)*(((U+V)>0)*U + ((U+V)<=0)*V) + (U<V)*((U>0)*U+(V<0)*V)
    return 0.5*ustar**2

def flux(U,V):
    #return flux_Godunov(U,V)
    return flux_upwind_conservative(U,V)
    #return flux_upwind_conservative_naive(U,V)

### Exercice 4:

- Modifiez la fonction de flux utilisée. Essayez d'utiliser la fonction flux_upwind_conservative_naive ou flux_Godunov plutôt que flux_upwind_conservative. 

Comment cela affecte-t-il la simulation ?

In [14]:
# Réponse



# Initialisation
- Initialisation de la densité en utilisant une fonction gaussienne **u**.

In [15]:
# Initial condition
# Fonction gaussienne
u = np.exp(-((xn-L/2)/(0.1*L))**2)+0.5


### Exercice 5:

- Modifiez la condition initiale pour utiliser une fonction de forme différente, comme une onde carrée ou une onde sinusoïdale. 

Comment cela affecte-t-il la simulation ?

In [16]:
# Réponse



# La boucle principale

In [17]:
c_anim = {}
anim_add_figure(u, t, c_anim)

# Main code
while t<tmax:
    # Update fields
    # Godunov
    F[1:Np] = flux(u[0:Np-1],u[1:Np])
    
    if Dirichlet: 
        F[0] = flux(u_g,u[0])
        F[Np] = flux(u[Np-1],u_d)
    elif Cyclic:
        F[Np] = flux(u[Np-1],u[0])
        F[0] = F[Np]
    
    dt = CFL * dx / np.max(u) # s
    u = u + dt/dx*(F[0:Np] - F[1:Np+1])

    t = t + dt
    
    # Plot
    if (abs(t % dtAff) < dt):
         anim_add_figure(u, t, c_anim)

my_animation(c_anim)

Cet animation qui montre l'évolution de la concentration du fluide.